# MVP 3 Scores

This file was mainly used for our final results for our dashboard

But this will show the featurs that we selected from the different clustering and imputation methods we used.

In [1]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

np.random.seed(42)

## Establishing Path

In [2]:
path = "../SHARED/"

# Recompletion

In [3]:
# These were the features that we chose as our best
pot_rcompl = [
    "Stage_Length (meters)",
    "STG_INITIAL_PUMPING_PRESSURE",
    "Prod_Per_Stage",
    "MAX_PUMP_RATING",
    "Rig_Duration (days)",
]

In [5]:
# Next read in all the different data for these features from
# the different clustering and imputation methods that we used
r_rf_ag = pd.read_excel(path + "PCA-RCOMPL-Agglomerative-RF.xlsx")
s_r_rf_ag = r_rf_ag[pot_rcompl]

r_rf_km = pd.read_excel(path + "PCA-RCOMPL-KMEANS-RF.xlsx")
s_r_rf_km = r_rf_km[pot_rcompl]

r_rf_sp = pd.read_excel(path + "PCA-RCOMPL-Spectral-RF.xlsx")
s_r_rf_sp = r_rf_sp[pot_rcompl]



r_knn_ag = pd.read_excel(path + "PCA-RCOMPL-Agglomerative-KNN.xlsx")
s_r_knn_ag = r_knn_ag[pot_rcompl]

r_knn_km = pd.read_excel(path + "PCA-RCOMPL-KMEANS-KNN.xlsx")
s_r_knn_km = r_knn_km[pot_rcompl]

r_knn_sp = pd.read_excel(path + "PCA-RCOMPL-Spectral-KNN.xlsx")
s_r_knn_sp = r_knn_sp[pot_rcompl]

# Make sure to include contractor to be able to distinguish what is what

r_rf_ag["CONTRACTOR"] = r_knn_ag["CONTRACTOR"]
r_rf_km["CONTRACTOR"] = r_knn_km["CONTRACTOR"] 
r_rf_sp["CONTRACTOR"] = r_knn_sp["CONTRACTOR"] 

In [6]:
# run all the files through scaling to go through the final grading process

scaler = MinMaxScaler()
s_r_rf_ag = pd.DataFrame(scaler.fit_transform(s_r_rf_ag), columns=s_r_rf_ag.columns)
s_r_rf_km = pd.DataFrame(scaler.fit_transform(s_r_rf_km), columns=s_r_rf_km.columns)
s_r_rf_sp = pd.DataFrame(scaler.fit_transform(s_r_rf_sp), columns=s_r_rf_sp.columns)

s_r_knn_ag = pd.DataFrame(scaler.fit_transform(s_r_knn_ag), columns=s_r_knn_ag.columns)
s_r_knn_km = pd.DataFrame(scaler.fit_transform(s_r_knn_km), columns=s_r_knn_km.columns)
s_r_knn_sp = pd.DataFrame(scaler.fit_transform(s_r_knn_sp), columns=s_r_knn_sp.columns)

In [7]:
# append the contractor data to the final result
s_r_rf_ag["CONTRACTOR"] = r_rf_ag["CONTRACTOR"]
s_r_rf_km["CONTRACTOR"] = r_rf_km["CONTRACTOR"]
s_r_rf_sp["CONTRACTOR"] = r_rf_sp["CONTRACTOR"]
s_r_knn_ag["CONTRACTOR"] = r_knn_ag["CONTRACTOR"]
s_r_knn_km["CONTRACTOR"] = r_knn_km["CONTRACTOR"]
s_r_knn_sp["CONTRACTOR"] = r_knn_sp["CONTRACTOR"]

In [8]:
# this will be extra stuff we may want to look at
extra = [
    "CONTRACTOR",
    "Cluster_method",
    "Imputation",
    "Final_score"
]

# Helps with organizing data
cluster_method = [
    "Agglomerative",
    "Kmeans",
    "Spectral"
]

imputation_method = [
    "RandomForest",
    "KNN"
]

all_files = [
    s_r_rf_ag,
    s_r_knn_ag,
    s_r_rf_km,
    s_r_knn_km,
    s_r_rf_sp,
    s_r_knn_sp
]

# Now making the dataframe that will hold all the results for recompletion
temp = pot_rcompl + extra
final_results_r = pd.DataFrame(columns = temp)

cluster = 0
imputation = 0
for f in all_files:
    unique_c = f["CONTRACTOR"].unique()
    
    unique_data = []
    for contractor in unique_c:
        unique_data.append(f[f["CONTRACTOR"] == contractor].reset_index())
    
    i = 0
    for data in unique_data:
        score_1 = data[pot_rcompl[0]].sum() / len(data)
        score_2 = data[pot_rcompl[1]].sum() / len(data)
        score_3 = data[pot_rcompl[2]].sum() / len(data)
        score_4 = data[pot_rcompl[3]].sum() / len(data)
        score_5 = data[pot_rcompl[4]].sum() / len(data)
        
        temp_row = {
            pot_rcompl[0]: score_1,
            pot_rcompl[1]: score_2,
            pot_rcompl[2]: score_3,
            pot_rcompl[3]: score_4,
            pot_rcompl[4]: score_5,
            extra[0]:      unique_c[i],
            extra[1]:      cluster_method[cluster//2],
            extra[2]:      imputation_method[imputation % 2],
            # Score will just be an evenly weighted average
            extra[3]:      0.2 * (score_1 + score_2 + score_3 + score_4 + score_5)
            
        }
        i += 1
        final_results_r = final_results_r.append(temp_row, ignore_index = True)
        
    cluster += 1
    imputation += 1
        

In [9]:
final_results_r

,Stage_Length (meters),STG_INITIAL_PUMPING_PRESSURE,Prod_Per_Stage,MAX_PUMP_RATING,Rig_Duration (days),CONTRACTOR,Cluster_method,Imputation,Final_score
0,0.510981,0.732721,0.148312,0.515181,0.116635,CONTRACTOR - 2019000027,Agglomerative,RandomForest,0.404766
1,0.379901,0.829216,0.175320,0.766519,0.078560,CONTRACTOR - 2019000030,Agglomerative,RandomForest,0.445903
2,0.067734,0.717821,0.068144,0.589595,0.012552,CONTRACTOR - 2019000023,Agglomerative,RandomForest,0.291169
3,0.154225,0.772998,0.139488,0.755397,0.037543,CONTRACTOR - 2019000008,Agglomerative,RandomForest,0.371930
4,0.751269,0.752382,0.402778,0.609493,0.110574,CONTRACTOR - 2019000051,Agglomerative,RandomForest,0.525299
5,0.501194,0.727795,0.128178,0.448821,0.126911,CONTRACTOR - 2019000027,Agglomerative,KNN,0.386580
6,0.167419,0.734325,0.122030,0.711463,0.021898,CONTRACTOR - 2019000030,Agglomerative,KNN,0.351427
7,0.750849,0.749089,0.282477,0.549707,0.119940,CONTRACTOR - 2019000023,Agglomerative,KNN,0.490413
8,0.365507,0.824144,0.159507,0.730189,0.080833,CONTRACTOR - 2019000008,Agglomerative,KNN,0.432036
9,0.051706,0.830752,0.068166,0.500000,0.028807,CONTRACTOR - 2019000051,Agglomerative,KNN,0.295886


In [8]:
final_results_r.to_excel(path + "Final_Scores_RCOMPL_MVP3.xlsx", index = False)

In [10]:
# Some other things that are going to be required for dashboard visualization
scores_to_take = ["PCA-X", "PCA-Y", "CLUSTER", "Cluster_method", "Imputation"]

temp = ["CONTRACTOR"] + scores_to_take + pot_rcompl

r_rf_ag["Cluster_method"] = cluster_method[0]
r_rf_ag["Imputation"] = imputation_method[0]
r_rf_ag = r_rf_ag[temp]

r_knn_ag["Cluster_method"] = cluster_method[0]
r_knn_ag["Imputation"] = imputation_method[1]
r_knn_ag = r_knn_ag[temp]

r_rf_km["Cluster_method"] = cluster_method[1]
r_rf_km["Imputation"] = imputation_method[0]
r_rf_km = r_rf_km[temp]

r_knn_km["Cluster_method"] = cluster_method[1]
r_knn_km["Imputation"] = imputation_method[1]
r_knn_km = r_knn_km[temp]

r_rf_sp["Cluster_method"] = cluster_method[2]
r_rf_sp["Imputation"] = imputation_method[0]
r_rf_sp = r_rf_sp[temp]

r_knn_sp["Cluster_method"] = cluster_method[2]
r_knn_sp["Imputation"] = imputation_method[1]
r_knn_sp = r_knn_sp[temp]


needed = [
    r_rf_ag,
    r_knn_ag,
    r_rf_km,
    r_knn_km,
    r_rf_sp,
    r_knn_sp,
]

pca_results_r = pd.DataFrame(columns = r_rf_ag.columns)

pca_results_r = pd.concat(needed)

In [11]:
pca_results_r

,CONTRACTOR,PCA-X,PCA-Y,CLUSTER,Cluster_method,Imputation,Stage_Length (meters),STG_INITIAL_PUMPING_PRESSURE,Prod_Per_Stage,MAX_PUMP_RATING,Rig_Duration (days)
0,CONTRACTOR - 2019000027,0.718164,1.062202,3,Agglomerative,RandomForest,4333,6575.0,2110.000000,30000.0,5
1,CONTRACTOR - 2019000027,0.904125,-0.131825,1,Agglomerative,RandomForest,4174,2964.0,0.000000,30000.0,3
2,CONTRACTOR - 2019000027,0.082686,0.062567,4,Agglomerative,RandomForest,100,5530.0,3213.380000,28000.0,9
3,CONTRACTOR - 2019000027,1.302956,-0.681224,1,Agglomerative,RandomForest,4236,7002.0,15079.000000,36000.0,124
4,CONTRACTOR - 2019000027,0.082746,0.062277,4,Agglomerative,RandomForest,100,5530.0,3213.380000,28000.0,9
...,...,...,...,...,...,...,...,...,...,...,...
1645,CONTRACTOR - 2019000051,-2.145201,-0.120392,1,Spectral,KNN,152,7618.0,2933.909091,32000.0,6
1646,CONTRACTOR - 2019000051,-2.136528,-0.158728,1,Spectral,KNN,152,7618.0,2933.909091,32000.0,6
1647,CONTRACTOR - 2019000051,-2.128159,-0.161731,1,Spectral,KNN,150,7618.0,2933.909091,32000.0,6
1648,CONTRACTOR - 2019000051,-2.125233,-0.113892,1,Spectral,KNN,151,7618.0,2933.909091,32000.0,6


In [11]:
pca_results_r.to_excel(path + "Final_PCA_RCOMPL_MVP3.xlsx", index = False)

## Stimulation stage

In [12]:
# Here are the features that we selected for our best.

pot_stmstg = [
    "PROPPANT_DESIGNED",
    "STG_SLURRY_VOLUME_PUMPED",
    "TREAT_MAX_PRESSURE",
    "MAX_PUMP_RATING",
    "INITIAL_SHUT_IN_PRESSURE",
]

In [13]:
# Reading in all the data aquired from running our different clustering methods

s_rf_ag = pd.read_excel(path + "PCA-STMSTG-Agglomerative-RF.xlsx")
s_s_rf_ag = s_rf_ag[pot_stmstg]

s_rf_km = pd.read_excel(path + "PCA-STMSTG-KMEANS-RF.xlsx")
s_s_rf_km = s_rf_km[pot_stmstg]

s_rf_sp = pd.read_excel(path + "PCA-STMSTG-Spectral-RF.xlsx")
s_s_rf_sp = s_rf_sp[pot_stmstg]

s_knn_ag = pd.read_excel(path + "PCA-STMSTG-Agglomerative-KNN.xlsx")
s_s_knn_ag = s_knn_ag[pot_stmstg]

s_knn_km = pd.read_excel(path + "PCA-STMSTG-KMEANS-KNN.xlsx")
s_s_knn_km = s_knn_km[pot_stmstg]

s_knn_sp = pd.read_excel(path + "PCA-STMSTG-Spectral-KNN.xlsx")
s_s_knn_sp = s_knn_sp[pot_stmstg]

s_rf_ag["CONTRACTOR"] = s_knn_ag["CONTRACTOR"]
s_rf_km["CONTRACTOR"] = s_knn_km["CONTRACTOR"] 
s_rf_sp["CONTRACTOR"] = s_knn_sp["CONTRACTOR"] 

In [14]:
# running it through a scaler to make values standardized
scaler = MinMaxScaler()
s_s_rf_ag = pd.DataFrame(scaler.fit_transform(s_s_rf_ag), columns=s_s_rf_ag.columns)
s_s_rf_km = pd.DataFrame(scaler.fit_transform(s_s_rf_km), columns=s_s_rf_km.columns)
s_s_rf_sp = pd.DataFrame(scaler.fit_transform(s_s_rf_sp), columns=s_s_rf_sp.columns)

s_s_knn_ag = pd.DataFrame(scaler.fit_transform(s_s_knn_ag), columns=s_s_knn_ag.columns)
s_s_knn_km = pd.DataFrame(scaler.fit_transform(s_s_knn_km), columns=s_s_knn_km.columns)
s_s_knn_sp = pd.DataFrame(scaler.fit_transform(s_s_knn_sp), columns=s_s_knn_sp.columns)

In [15]:
# Including contractor to distinguish who is who
s_s_rf_ag["CONTRACTOR"] = s_rf_ag["CONTRACTOR"]
s_s_rf_km["CONTRACTOR"] = s_rf_km["CONTRACTOR"]
s_s_rf_sp["CONTRACTOR"] = s_rf_sp["CONTRACTOR"]
s_s_knn_ag["CONTRACTOR"] = s_knn_ag["CONTRACTOR"]
s_s_knn_km["CONTRACTOR"] = s_knn_km["CONTRACTOR"]
s_s_knn_sp["CONTRACTOR"] = s_knn_sp["CONTRACTOR"]

In [16]:
# Extra info to help with dashboard visualization
extra = [
    "CONTRACTOR",
    "Cluster_method",
    "Imputation",
    "Final_score"
]

# Helps with how we label data
cluster_method = [
    "Agglomerative",
    "Kmeans",
    "Spectral"
]

imputation_method = [
    "RandomForest",
    "KNN"
]

all_files = [
    s_s_rf_ag,
    s_s_knn_ag,
    s_s_rf_km,
    s_s_knn_km,
    s_s_rf_sp,
    s_s_knn_sp
]

# Constructing the final results for the features we selected
temp = pot_stmstg + extra
final_results_s = pd.DataFrame(columns = temp)

cluster = 0
imputation = 0
for f in all_files:
    unique_c = f["CONTRACTOR"].unique()
    
    unique_data = []
    for contractor in unique_c:
        unique_data.append(f[f["CONTRACTOR"] == contractor].reset_index())
    
    i = 0
    for data in unique_data:
        score_1 = data[pot_stmstg[0]].sum() / len(data)
        score_2 = data[pot_stmstg[1]].sum() / len(data)
        score_3 = data[pot_stmstg[2]].sum() / len(data)
        score_4 = data[pot_stmstg[3]].sum() / len(data)
        score_5 = data[pot_stmstg[4]].sum() / len(data)
        
        temp_row = {
            pot_stmstg[0]: score_1,
            pot_stmstg[1]: score_2,
            pot_stmstg[2]: score_3,
            pot_stmstg[3]: score_4,
            pot_stmstg[4]: score_5,
            extra[0]:      unique_c[i],
            extra[1]:      cluster_method[cluster//2],
            extra[2]:      imputation_method[imputation % 2],
            extra[3]:      0.2 * (score_1 + score_2 + score_3 + score_4 + score_5)
            
        }
        i += 1
        final_results_s = final_results_s.append(temp_row, ignore_index = True)
        
    cluster += 1
    imputation += 1
    print("Finished")

Finished
Finished
Finished
Finished
Finished
Finished


In [17]:
final_results_s

,PROPPANT_DESIGNED,STG_SLURRY_VOLUME_PUMPED,TREAT_MAX_PRESSURE,MAX_PUMP_RATING,INITIAL_SHUT_IN_PRESSURE,CONTRACTOR,Cluster_method,Imputation,Final_score
0,0.300901,0.192612,0.094597,0.059377,0.514912,CONTRACTOR - 2019000008,Agglomerative,RandomForest,0.232480
1,0.318800,0.203731,0.096943,0.067634,0.518939,CONTRACTOR - 2019000027,Agglomerative,RandomForest,0.241209
2,0.288010,0.160572,0.093021,0.057751,0.403031,CONTRACTOR - 2019000030,Agglomerative,RandomForest,0.200477
3,0.236881,0.153726,0.094314,0.084500,0.492520,CONTRACTOR - 2019000023,Agglomerative,RandomForest,0.212388
4,0.331998,0.218649,0.093972,0.063879,0.525614,CONTRACTOR - 2019000051,Agglomerative,RandomForest,0.246822
5,0.362890,0.204424,0.818123,0.630064,0.518685,CONTRACTOR - 2019000008,Agglomerative,KNN,0.506837
6,0.340816,0.193677,0.797702,0.582300,0.518121,CONTRACTOR - 2019000027,Agglomerative,KNN,0.486523
7,0.271247,0.155760,0.791891,0.767887,0.491119,CONTRACTOR - 2019000030,Agglomerative,KNN,0.495581
8,0.375764,0.216685,0.794459,0.602957,0.527901,CONTRACTOR - 2019000023,Agglomerative,KNN,0.503553
9,0.328231,0.162371,0.783433,0.554343,0.404612,CONTRACTOR - 2019000051,Agglomerative,KNN,0.446598


In [18]:
# Last additional features needed for dashboard visualization
scores_to_take = ["PCA-X", "PCA-Y", "CLUSTER", "Cluster_method", "Imputation"]

temp = ["CONTRACTOR"] + scores_to_take + pot_stmstg


s_rf_ag["Cluster_method"] = cluster_method[0]
s_rf_ag["Imputation"] = imputation_method[0]
s_rf_ag = s_rf_ag[temp]

s_knn_ag["Cluster_method"] = cluster_method[0]
s_knn_ag["Imputation"] = imputation_method[1]
s_knn_ag = s_knn_ag[temp]

s_rf_km["Cluster_method"] = cluster_method[1]
s_rf_km["Imputation"] = imputation_method[0]
s_rf_km = s_rf_km[temp]

s_knn_km["Cluster_method"] = cluster_method[1]
s_knn_km["Imputation"] = imputation_method[1]
s_knn_km = s_knn_km[temp]

s_rf_sp["Cluster_method"] = cluster_method[2]
s_rf_sp["Imputation"] = imputation_method[0]
s_rf_sp = s_rf_sp[temp]

s_knn_sp["Cluster_method"] = cluster_method[2]
s_knn_sp["Imputation"] = imputation_method[1]
s_knn_sp = s_knn_sp[temp]


needed = [
    s_rf_ag,
    s_knn_ag,
    s_rf_km,
    s_knn_km,
    s_rf_sp,
    s_knn_sp,
]

pca_results_s = pd.DataFrame(columns = r_rf_ag.columns)

pca_results_s = pd.concat(needed)
pca_results_s

,CONTRACTOR,PCA-X,PCA-Y,CLUSTER,Cluster_method,Imputation,PROPPANT_DESIGNED,STG_SLURRY_VOLUME_PUMPED,TREAT_MAX_PRESSURE,MAX_PUMP_RATING,INITIAL_SHUT_IN_PRESSURE
0,CONTRACTOR - 2019000008,0.506718,0.249234,4,Agglomerative,RandomForest,260750,5545,7489.0,30000.0,6681.0
1,CONTRACTOR - 2019000008,0.028986,-0.362479,2,Agglomerative,RandomForest,420000,9146,10143.0,36000.0,7245.0
2,CONTRACTOR - 2019000008,-0.141952,1.195822,0,Agglomerative,RandomForest,420000,7473,11128.0,34000.0,7282.0
3,CONTRACTOR - 2019000008,-0.434382,0.054774,0,Agglomerative,RandomForest,426125,9297,10270.0,36000.0,5110.0
4,CONTRACTOR - 2019000008,0.426761,1.072818,0,Agglomerative,RandomForest,420000,5658,10584.0,32000.0,4989.0
...,...,...,...,...,...,...,...,...,...,...,...
17715,CONTRACTOR - 2019000051,-1.682751,-0.040359,0,Spectral,KNN,420000,5896,10458.0,32000.0,4813.0
17716,CONTRACTOR - 2019000051,-1.624385,0.030330,0,Spectral,KNN,420000,11313,10207.0,31600.0,4191.0
17717,CONTRACTOR - 2019000051,-1.537912,-0.111868,0,Spectral,KNN,420000,8141,10151.0,32000.0,6220.0
17718,CONTRACTOR - 2019000051,-1.605242,-0.110533,3,Spectral,KNN,420000,7056,10420.0,32000.0,5266.0


In [20]:
pca_results_s.to_excel(path + "Final_PCA_STMSTG_MVP3.xlsx", index = False)

In [21]:
final_results_s.to_excel(path + "Final_Scores_STMSTG_MVP3.xlsx", index = False)